<a href="https://colab.research.google.com/github/pablohenrique93/projeto-final-intro-machine-learning-python/blob/main/final_project_machine_learning_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação das Bibliotecas

In [ ]:
#Instalação da biblioteca para pré-processamento de dados 
pip install pandera

In [ ]:
#Instalação do plotly para visualização de dados
pip install plotly --upgrade

In [3]:
# Instalação do yellowbrick para trabalhar com Machine Learning
!pip -q install yellowbrick

#Importação das Bibliotecas

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#Exploração dos Dados

In [5]:
base_predictive = pd.read_csv('/content/predictive_maintenance.csv')

In [ ]:
#Visualizando o dataframe
base_predictive.head()

In [ ]:
base_predictive.tail()

In [ ]:
#Verificando os tipos das colunas
base_predictive.dtypes

In [ ]:
#Verificando o tamanho da base
base_predictive.shape

#Tratamento dos Dados

In [ ]:
pd.unique(base_predictive['Product ID'])

In [ ]:
pd.unique(base_predictive['Type'])

In [ ]:
pd.unique(base_predictive['Air temperature [K]'])

In [ ]:
pd.unique(base_predictive['Process temperature [K]'])

In [ ]:
pd.unique(base_predictive['Rotational speed [rpm]'])

In [ ]:
pd.unique(base_predictive['Torque [Nm]'])

In [ ]:
pd.unique(base_predictive['Tool wear [min]'])

In [ ]:
pd.unique(base_predictive['Target']) 

In [ ]:
pd.unique(base_predictive['Failure Type']) 

#Limpeza de Inconsistências e Padronização de Colunas

In [ ]:
base_predictive.dtypes

In [19]:
#Renomeação e tradução das colunas com o objetivo de facilitar o entendimento dos dados
base_predictive.rename(columns={
    'UDI':'id',
    'Product ID':'qualidade_produto',
    'Type':'tipo',
    'Air temperature [K]':'temperatura_ar_k',
    'Process temperature [K]':'temperatura_processo_k',
    'Rotational speed [rpm]':'velocidade_rotacao_rpm',
    'Torque [Nm]':'torque_nm',
    'Tool wear [min]':'desgaste_ferramenta_min',
    'Target':'falha_ou_nao',
    'Failure Type':'tipo_falha'
},inplace=True)

In [20]:
#Traduzindo dados dentro da coluna "tipo falha", para facilitar o entendimento
base_predictive.replace({'tipo_falha':'No Failure'}, 'Sem Falhas',regex=True, inplace=True)
base_predictive.replace({'tipo_falha':'Power Failure'}, 'Falha de Energia',regex=True, inplace=True)
base_predictive.replace({'tipo_falha':'Tool Wear Failure'}, 'Falha no Desgaste de Ferramentas',regex=True, inplace=True)
base_predictive.replace({'tipo_falha':'Overstrain Failure'}, 'Falha de Sobrecarga',regex=True, inplace=True)
base_predictive.replace({'tipo_falha':'Random Failures'}, 'Falhas Aleatórias',regex=True, inplace=True)
base_predictive.replace({'tipo_falha':'Heat Dissipation Failure'}, 'Falha na Dissipação de Calor',regex=True, inplace=True)

In [ ]:
#Conferindo coluna pós manipulação
pd.unique(base_predictive['tipo_falha'])

Verificação de Dados Faltantes

In [ ]:
base_predictive.isnull()

In [ ]:
#Como este dataframe é grande, faz-se necessário a utilização deste código para verificar valores nulos
base_predictive.isnull().sum()
#Como podemos observar não existe valores nulos neste dataframe

#Visualização dos Dados

In [ ]:
#Código para exibir os registros da coluna 'falha_ou_nao' assim como quantas vezes os mesmos aparecerem
np.unique(base_predictive['falha_ou_nao'], return_counts=True)

In [ ]:
# 0 NÃO HOUVE FALHA
# 1 HOUVE FALHA 
#Plot utilizando seaborn para visualização dos registros da coluna 'falha_ou_nao'
sns.countplot(x=base_predictive['falha_ou_nao']); 

In [ ]:
#Código para exibir os registros da coluna 'tipo falha' assim como quantas vezes os mesmos ocorrem
np.unique(base_predictive['tipo_falha'], return_counts=True)

In [ ]:
#Visualização de registros da coluna 'tipo_falha' (OBS: é possível notar que a maioria não sofreu falhas) 
plt.hist(x=base_predictive['tipo_falha']);

In [ ]:
#Visualização de registros da coluna 'tipo', constituída por uma letra L, M, ou H, para (L) baixo 
#(50% de todos os produtos), (M) médio (30%), e (H) alto (20%) como variantes de qualidade do produto
plt.hist(x=base_predictive['tipo']);

In [ ]:
#Visualização dos índices de temperatura de processo
plt.hist(x=base_predictive['temperatura_processo_k']);

In [ ]:
#Visualização dos índices de velocidade de rotação 
plt.hist(x=base_predictive['velocidade_rotacao_rpm']);

#Divisão dos Previsores e Classes

PREVISORES são as variáveis que vão ser utilizadas para fazer as previsões

CLASSES: São os objetivos daquilo que queremos prever

In [ ]:
base_predictive

In [33]:
#Previsores
X_base_predictive = base_predictive.iloc[:, 3:8].values

In [ ]:
#Conferindo o novo dataframe/Previsores
X_base_predictive

In [35]:
#Classes
Y_base_predictive = base_predictive.iloc[:, 8].values

In [ ]:
#Conferindo o novo dataframe/Classe
Y_base_predictive

In [ ]:
#Conferindo e comparando os tipos de cada dataframe
type(base_predictive)

In [ ]:
#Os dataframes que serão trabalhados serão arrays de Numpy
type(X_base_predictive)
type(Y_base_predictive)

#Escalonamento de Valores

In [ ]:
X_base_predictive

In [ ]:
#Neste comando pegamos o menor elemento das colunas seguido dos seus índices
X_base_predictive[:, 0].min(), X_base_predictive[:, 1].min(), X_base_predictive[:, 2].min(), X_base_predictive[:, 3].min(), X_base_predictive[:, 4].min()
#É possível notar que os valores estão bem desbalanceados

In [ ]:
#A mesma situação ocorre com valores máximos
X_base_predictive[:, 0].max(), X_base_predictive[:, 1].max(), X_base_predictive[:, 2].max(), X_base_predictive[:, 3].max(), X_base_predictive[:, 4].max()

Dois modelos de escalonamento

1 - Padronização - 
Utilizado quando existe um desvio muito grande de valores da média x = x - media(x)/desvio padrão(x)

2- Normalização -
Quando temos diferenças muito pequenas nos valores e queremos que os valores se comportem da mesma maneira x = x - mínimo(x)/máximo(x) - mínimo(x)

In [42]:
#Neste caso utilizaremos a Padronização devido diferença grande entre os valores, como observado

In [43]:
# Para realizar esta operação utilizamos o StandardScaler
from sklearn.preprocessing import StandardScaler

In [44]:
#Instanciando a biblioteca e posteriormente padronizando o X_base_predictive (Previsor)
scaler_credit = StandardScaler()
X_base_predictive = scaler_credit.fit_transform(X_base_predictive)

In [ ]:
X_base_predictive

In [ ]:
#Verificando os valores que já foram aproximados
X_base_predictive[:, 0].min(), X_base_predictive[:, 1].min(), X_base_predictive[:, 2].min(), X_base_predictive[:, 3].min(), X_base_predictive[:, 4].min()

In [ ]:
X_base_predictive[:, 0].max(), X_base_predictive[:, 1].max(), X_base_predictive[:, 2].max(), X_base_predictive[:, 3].max(), X_base_predictive[:, 4].max()

In [ ]:
X_base_predictive

# Divisão de Bases em Treinamento e Teste

Os algaritmos de Machine Learning precisam de uma base de treinamento e uma base para testar nossas previsões.\n
OBS: ELAS NÃO PODEM SER IGUAIS

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
X_predictive_treinamento, X_predictive_teste, Y_predictive_treinamento, Y_predictive_teste = train_test_split(X_base_predictive, Y_base_predictive, test_size = 0.25, random_state = 0)

In [ ]:
#Conferindo as bases
X_predictive_treinamento.shape, Y_predictive_treinamento.shape

In [ ]:
X_predictive_teste.shape, Y_predictive_teste.shape

# Salvando as Variáveis

In [53]:
import pickle

In [ ]:
#Salvando as variáveis da base em um ficheiro
with open('predictive.pkl', mode = 'wb') as f:
  pickle.dump([X_predictive_treinamento, Y_predictive_treinamento, X_predictive_teste, Y_predictive_teste], f)

# Algoritmo de Naive Bayes

In [54]:
#Através deste algoritmo, verificaremos com base em características se houve falha ou não na máquina

In [55]:
#Importação da biblioteca necessária
from sklearn.naive_bayes import GaussianNB

In [56]:
#Fazendo o carregamento dos arquivos salvos
with open('/content/predictive.pkl', 'rb') as f:
  X_predictive_treinamento, Y_predictive_treinamento, X_predictive_teste, Y_predictive_teste = pickle.load(f)

In [ ]:
#Verificando se os arquivos foram devidamente carregados
X_predictive_treinamento.shape, Y_predictive_treinamento.shape

In [ ]:
X_predictive_teste.shape, Y_predictive_teste.shape

In [ ]:
#Criação do algoritmo para fazer as previsões
naive_predictive = GaussianNB()
naive_predictive.fit(X_predictive_treinamento, Y_predictive_treinamento)

In [60]:
#Realização das Previsões
previsoes = naive_predictive.predict(X_predictive_teste)

In [ ]:
#Previsoes apresentadas
previsoes

In [ ]:
#Em comparação com o resultado anterior, é possível notar que o algoritmo conseguiu prever a maioria dos resultados,
#retornando os valores idênticos ao original, onde não houveram falhas nas máquinas, indicando uma taxa alta de Acerto
Y_predictive_teste

In [63]:
#Importação da biblioteca para medição precisão
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
#Com este código é possível fazer a associação entre o original e a previsão realizada
accuracy_score(Y_predictive_teste, previsoes)
#É possivel notar que a precisão é de 96%, ou seja, bem alta

In [ ]:
confusion_matrix(Y_predictive_teste, previsoes)
#Como podemos observar, é possível verificar que 2387 máquinas não sofrem falhas e 25 sofrem
#E de 72 máquinas que sofrem falhas 16 foram classificadas que sofrem falhas

In [66]:
#Importação da biblioteca para conferência desta situação de maneira visual (Conferir no final deste caderno)
from yellowbrick.classifier import ConfusionMatrix

#Algoritmo de Árvore de Decisão (Decision Tree)

Esse algoritmo cria uma árvore com vários "nós" onde ele faz algumas buscas de acordo com algumas características que ele encontra

In [67]:
#Importação da biblioteca
from sklearn.tree import DecisionTreeClassifier

In [68]:
#Fazendo o carregamento dos arquivos salvos
with open('/content/predictive.pkl', 'rb') as f:
  X_predictive_treinamento, Y_predictive_treinamento, X_predictive_teste, Y_predictive_teste = pickle.load(f)

In [ ]:
#Verificando se os arquivos foram devidamente carregados
X_predictive_treinamento.shape, Y_predictive_treinamento.shape

In [ ]:
X_predictive_teste.shape, Y_predictive_teste.shape

In [71]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_predictive_treinamento = le.fit_transform(Y_predictive_treinamento)

In [ ]:
#Criação da árvore de decisão
# 'criterion='entropy' - Variável utilizada para fazer cálculo matemático
#Parte da premissa que toda informação que vai percorrer algum canal vai sofrer alguma alteração isso se chama entropia
arvore_predictive = DecisionTreeClassifier(criterion='entropy', random_state=0)
arvore_predictive.fit(X_predictive_treinamento, Y_predictive_treinamento)

In [ ]:
#Exibindo valores que o algoritmo julga ser importante para realizar os cálculos
arvore_predictive.feature_importances_

In [ ]:
#Exibindo as classes que o algoritmo vai utilizar como parâmetro
arvore_predictive.classes_

In [ ]:
#Criando previsões
previsoes2 = arvore_predictive.predict(X_predictive_teste)
previsoes2

In [ ]:
Y_predictive_teste
#Como podemos observar o algoritmo foi bastante acertivo, retornando previsões contendo os mesmos dados originais

In [ ]:
confusion_matrix(Y_predictive_teste, previsoes2)
#Como podemos observar, é possível verificar que 2385 máquinas não sofreram falhas e somente 27 foram classificados como que "sofreram falhas"
#E de 31 máquinas "que sofreram falhas" 57 foram classificadas "que sofrem falhas"/ Como verificado este algoritmo é bastante eficiente

In [78]:
#Medindo a previsão
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(Y_predictive_teste, previsoes2)
#Como verificado, 97% de confiabilidade, ou seja uma precisão bastante elevada

# Algoritmo de Random Forest (Floresta Randômica)

Este algoritmo é uma extensão da Árvore de Decisão, ele se caracteriza como um conjunto de árvores de decisão

In [80]:
#Importando a biblioteca
from sklearn.ensemble import RandomForestClassifier

In [81]:
import pickle
with open('/content/predictive.pkl', 'rb') as f:  
  X_predictive_treinamento, Y_predictive_treinamento, X_predictive_teste, Y_predictive_teste = pickle.load(f)

In [ ]:
#Conferindo se arquivos foram devidamente carregados
X_predictive_treinamento.shape, Y_predictive_treinamento.shape

In [ ]:
X_predictive_teste.shape, Y_predictive_teste.shape

In [84]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_predictive_treinamento = le.fit_transform(Y_predictive_treinamento)

In [ ]:
#Criando a Floresta
#n_estimators é o número de árvores que desejamos criar
from scipy.sparse.construct import random
random_forest_predictive = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state = 0)
random_forest_predictive.fit(X_predictive_treinamento, Y_predictive_treinamento)

In [ ]:
#Criando previsões
previsoes3 = random_forest_predictive.predict(X_predictive_teste)
previsoes3

In [ ]:
Y_predictive_teste
#Novamente possível notar uma confiabilidade bastante acertiva, onde o algoritmo faz as previsões de forma correta

In [ ]:
confusion_matrix(Y_predictive_teste, previsoes3)
#Como podemos observar, é possível verificar que 2401 máquinas não sofreram falhas e somente 11 foram classificados como que "sofreram falhas"
#E de 34 máquinas "que sofreram falhas" 54 foram classificadas "que sofrem falhas"/ Como verificado este algoritmo é bastante eficiente

In [ ]:
#Medindo nível de confiabilidade
from sklearn.metrics import accuracy_score
accuracy_score(Y_predictive_teste, previsoes3)

# Conclusão

In [ ]:
'''
Como visto anteriormente, neste projeto foram utilizados 3 algoritmos de Machine Learning sendo eles: Algoritmo de Naive Bayes, Decision Tree,
e Random Forest, cada um com suas particularidades, de modo geral a aplicação destes algoritmos nesta base de dados foi bastante eficiente,
retornando valores fiés aos dados da base original, todos três com alto índice de confiabilidade, logo abaixo é possível notar que o Algoritmo
Random Forest foi o mais preciso dos 3, com 98% de nível de confiabilidade, em segundo lugar o Algoritmo Decision Tree com 97% e por ultimo e 
não menos importante o Algoritmo de Naive Bayes com 96% 

Logo abaixo é possível verificar os algoritmos por ordem de precisão e os seus respectivos resultados, valendo ressaltar que todos os três 
trouxeram resultados bastante precisos.


'''

In [ ]:
# Random Forest (Floresta Randômica)
from yellowbrick.classifier import ConfusionMatrix
cm = ConfusionMatrix(random_forest_predictive)
cm.fit(X_predictive_treinamento, Y_predictive_treinamento)
cm.score(X_predictive_teste, Y_predictive_teste)

In [ ]:
'''
Como podemos observar, é possível verificar que 2401 máquinas não sofreram falhas e somente 11 foram classificados como que 'sofreram falhas'
E de 34 máquinas "que sofreram falhas" 54 foram classificadas que 'sofrem falhas'
'''

In [ ]:
#Decision Tree (Árvore de decisão)
from yellowbrick.classifier import ConfusionMatrix
cm = ConfusionMatrix(arvore_predictive)
cm.fit(X_predictive_treinamento, Y_predictive_treinamento)
cm.score(X_predictive_teste, Y_predictive_teste)

In [ ]:
'''
É possível verificar que 2385 máquinas não sofreram falhas e somente 27 foram classificados como que "sofreram falhas"
E de 31 máquinas "que sofreram falhas" 57 foram classificadas "que sofrem falhas".
'''

In [ ]:
#Algoritmo de Naive Bayes
cm = ConfusionMatrix(naive_predictive)
cm.fit(X_predictive_treinamento, Y_predictive_treinamento)
cm.score(X_predictive_teste, Y_predictive_teste)

In [ ]:
'''
Como podemos observar, é possível verificar que 2387 máquinas não sofrem falhas e 25 sofrem
E de 72 máquinas que sofrem falhas 16 foram classificadas que "sofrem falhas"
'''